In [ ]:
import os
import tensorflow
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [ ]:
from google.colab import files
files.upload()
#upload the kaggle.json file

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"princejohn","key":"9eb4d785a04ce184120453dd2e096246"}'}

In [ ]:
!mkdir -p ~/.kaggle  
!cp kaggle.json ~/.kaggle 
!chmod 600 ~/.kaggle/kaggle.json 

In [ ]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

  0% 0.00/450k [00:00<?, ?B/s]
100% 450k/450k [00:00<00:00, 30.6MB/s]
  0% 0.00/31.2k [00:00<?, ?B/s]
100% 31.2k/31.2k [00:00<00:00, 27.2MB/s]
  0% 0.00/13.1k [00:00<?, ?B/s]
100% 13.1k/13.1k [00:00<00:00, 12.0MB/s]
  0% 0.00/441k [00:00<?, ?B/s]
100% 441k/441k [00:00<00:00, 55.7MB/s]


In [ ]:
# Training data provided by Kaggle
df=pd.read_csv('/content/train.csv',na_values=['NA','?'])
# Testing data provided by Kaggle (public)
test=pd.read_csv('/content/test.csv',na_values=['NA','?'])

In [ ]:
ID = test.Id

In [ ]:
(df.isnull().sum()/df.shape[0]*100).sort_values(ascending=False)[0:29]

PoolQC          99.520548
MiscFeature     96.301370
Alley           93.767123
Fence           80.753425
FireplaceQu     47.260274
LotFrontage     17.739726
GarageCond       5.547945
GarageType       5.547945
GarageYrBlt      5.547945
GarageFinish     5.547945
GarageQual       5.547945
BsmtExposure     2.602740
BsmtFinType2     2.602740
BsmtFinType1     2.534247
BsmtCond         2.534247
BsmtQual         2.534247
MasVnrArea       0.547945
MasVnrType       0.547945
Electrical       0.068493
Utilities        0.000000
YearRemodAdd     0.000000
MSSubClass       0.000000
Foundation       0.000000
ExterCond        0.000000
ExterQual        0.000000
Exterior2nd      0.000000
Exterior1st      0.000000
RoofMatl         0.000000
RoofStyle        0.000000
dtype: float64

In [ ]:
# Imputing the median into the other features with missing values.
df["PoolQC"] = df["PoolQC"].fillna("None")
df["MiscFeature"] = df["MiscFeature"].fillna("None")
df["Fence"] = df["Fence"].fillna("None")
df["Alley"] = df["Alley"].fillna("None")
df["Alley"] = df["Alley"].fillna("None")
df["FireplaceQu"] = df["FireplaceQu"].fillna("None")
df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

#Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    df[col] = df[col].fillna('None')
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    df[col] = df[col].fillna(0)
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    df[col] = df[col].fillna(0)
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    df[col] = df[col].fillna('None')

df["MasVnrType"] = df["MasVnrType"].fillna("None")
df["MasVnrArea"] = df["MasVnrArea"].fillna(0)
df['MSZoning'] = df['MSZoning'].fillna(df['MSZoning'].mode()[0])
df = df.drop(['Utilities'], axis=1)
#df=df.drop(['Fence'], axis=1)
df["Functional"] = df["Functional"].fillna("Typ")
df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])
df['KitchenQual'] = df['KitchenQual'].fillna(df['KitchenQual'].mode()[0])
df['Exterior1st'] = df['Exterior1st'].fillna(df['Exterior1st'].mode()[0])
df['Exterior2nd'] = df['Exterior2nd'].fillna(df['Exterior2nd'].mode()[0])
df['SaleType'] = df['SaleType'].fillna(df['SaleType'].mode()[0])
df['MSSubClass'] = df['MSSubClass'].fillna("None")

#MSSubClass=The building class
df['MSSubClass'] = df['MSSubClass'].apply(float)


#Changing OverallCond into a categorical variable
df['OverallCond'] = df['OverallCond'].astype(float)


#Year and month sold are transformed into categorical features.
df['YrSold'] = df['YrSold'].astype(float)
df['MoSold'] = df['MoSold'].astype(float)


In [ ]:
df = pd.get_dummies(data=df, drop_first=True)

In [ ]:
display(df)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,MSZoning_FV,MSZoning_RH,...,GarageType_None,GarageFinish_None,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_None,GarageQual_Po,GarageQual_TA,GarageCond_Fa,GarageCond_Gd,GarageCond_None,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y,PoolQC_Fa,PoolQC_Gd,PoolQC_None,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,MiscFeature_None,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60.0,65.0,8450,7,5.0,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2.0,2008.0,208500,0,0,...,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1,2,20.0,80.0,9600,6,8.0,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5.0,2007.0,181500,0,0,...,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
2,3,60.0,68.0,11250,7,5.0,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9.0,2008.0,223500,0,0,...,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
3,4,70.0,60.0,9550,7,5.0,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2.0,2006.0,140000,0,0,...,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,5,60.0,84.0,14260,8,5.0,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12.0,2008.0,250000,0,0,...,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60.0,62.0,7917,6,5.0,1999,2000,0.0,0,0,953,953,953,694,0,1647,0,0,2,1,3,1,7,1,1999.0,2,460,0,40,0,0,0,0,0,8.0,2007.0,175000,0,0,...,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1456,1457,20.0,85.0,13175,6,6.0,1978,1988,119.0,790,163,589,1542,2073,0,0,2073,1,0,2,0,3,1,7,2,1978.0,2,500,349,0,0,0,0,0,0,2.0,2010.0,210000,0,0,...,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1457,1458,70.0,66.0,9042,7,9.0,1941,2006,0.0,275,0,877,1152,1188,1152,0,2340,0,0,2,0,4,1,9,2,1941.0,1,252,0,60,0,0,0,0,2500,5.0,2010.0,266500,0,0,...,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1458,1459,20.0,68.0,9717,5,6.0,1950,1996,0.0,49,1029,0,1078,1078,0,0,1078,1,0,1,0,2,1,5,0,1950.0,1,240,366,0,112,0,0,0,0,4.0,2010.0,142125,0,0,...,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0


In [ ]:
x = df.drop(['SalePrice'], 1)
y = df['SalePrice']

from sklearn.preprocessing import StandardScaler


# Feature scaling of our data
#sc = StandardScaler()
#x = pd.DataFrame(sc.fit_transform(x.values), index=x.index, columns=x.columns)
#x.head()

In [ ]:
import os
import tensorflow
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.33,random_state=42)


In [ ]:

model = Sequential()
model.add(Dense(200, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(100, kernel_initializer='normal', activation='relu'))
model.add(Dense(50, kernel_initializer='normal', activation='relu'))
model.add(Dense(25, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.compile(loss='mean_squared_error', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-4, 
                        patience=5, verbose=1, mode='auto', 
                        restore_best_weights=True)
#model.fit(x_train,y_train,callbacks=[monitor],verbose=2,epochs=1000)
model.fit(x_train, y_train, epochs=1000, validation_data=(x_test,y_test),batch_size=10,verbose=2)



Epoch 1/1000
98/98 - 1s - loss: 24835024896.0000 - val_loss: 5801272832.0000
Epoch 2/1000
98/98 - 0s - loss: 7011611648.0000 - val_loss: 5541473280.0000
Epoch 3/1000
98/98 - 0s - loss: 4841397248.0000 - val_loss: 4456975360.0000
Epoch 4/1000
98/98 - 0s - loss: 4010545664.0000 - val_loss: 4022264832.0000
Epoch 5/1000
98/98 - 0s - loss: 3514839808.0000 - val_loss: 3610389760.0000
Epoch 6/1000
98/98 - 0s - loss: 3191324672.0000 - val_loss: 3420585216.0000
Epoch 7/1000
98/98 - 0s - loss: 3009472000.0000 - val_loss: 3345734656.0000
Epoch 8/1000
98/98 - 0s - loss: 2753990144.0000 - val_loss: 3001047296.0000
Epoch 9/1000
98/98 - 0s - loss: 2566204416.0000 - val_loss: 3097096192.0000
Epoch 10/1000
98/98 - 0s - loss: 2436707328.0000 - val_loss: 2618259456.0000
Epoch 11/1000
98/98 - 0s - loss: 2378005504.0000 - val_loss: 2428515840.0000
Epoch 12/1000
98/98 - 0s - loss: 2233499904.0000 - val_loss: 2242472192.0000
Epoch 13/1000
98/98 - 0s - loss: 2245721600.0000 - val_loss: 2141572864.0000
Epoch 1

In [ ]:
# RMSE
from sklearn import metrics
#Predict
pred=model.predict(x_test)
score=np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final Score: {}".format(score))
pred.shape

Final Score: 42062.73220458929


(482, 1)